In [0]:
%tensorflow_version 1.x

import numpy as np
import keras
from keras.datasets import mnist
import sys
from scipy.stats import entropy
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, SpatialDropout2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.regularizers import l2
from keras import backend as K

from google.colab import drive
drive.mount("/content/gdrive")

TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean and variance of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns
        Mean and variance of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0]))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:] = predictions.flatten()
    prediction = result.mean(axis=0)
    var = result.var(axis=0)
    return [prediction,var]

def run_model (X_train, y_train):
    """Initializes and trains a model from scratch on the given training data, returning test MAE and MSE.
    #Arguments
        X_train: training model inputs
        y_train: training model outputs
    #Returns
        Test MAE and MSE
    """
    nb_pool = 3
    dropout_prob = 0.5
    nb_filters = 50
    nb_conv_init = 4 
    nb_conv_agg = 3
    nb_strides_init = 3
    nb_strides_agg = 2
    img_rows = 200
    img_cols = 200

    #Setting weight decay proportional to 1/N, where N is # training points
    Weight_Decay = 2.0/len(y_train)

    #Model 
    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv_init,  strides=nb_strides_agg, data_format="channels_last", input_shape=(img_rows, img_cols,3)))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, nb_conv_agg, strides=nb_strides_agg, data_format="channels_last"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_last"))

    model.add(Convolution2D(nb_filters*2, nb_conv_agg, strides=1, data_format="channels_last"))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters*2, nb_conv_agg, strides=1, data_format="channels_last"))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool), data_format="channels_last"))
    model.add(Dropout(dropout_prob))

    model.add(Flatten())
    model.add(Dense(150, W_regularizer=l2(Weight_Decay)))
    model.add(Activation('relu'))
    model.add(Dropout(dropout_prob))
    model.add(Dense(1, W_regularizer=l2(Weight_Decay)))
    model.compile(loss='mean_squared_error', optimizer='adam')

    #Training and evaluating on test data
    hist = model.fit(X_train, y_train, batch_size=128, epochs=250, verbose=0)
    f = K.function([model.layers[0].input, K.learning_phase()], 
               [model.layers[-1].output])
    prediction_results = predict_with_uncertainty(f, X_test, n_iter=300)
    predicted_mean = prediction_results[0]
    return [np.mean(np.abs(predicted_mean - y_test)), np.mean(np.square(predicted_mean - y_test))]


In [0]:
#Data Loading 
data_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/UTKFace/"
trind_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/UTKFace/ALScripts_StartTrainSize150/"

X_train_all = np.load(data_path+"X_train.npy")
y_train_all = np.load(data_path+"y_train.npy")
X_test = np.load(data_path+"X_test.npy")
y_test = np.load(data_path+"y_test.npy")


In [0]:
#Active learning settings/parameters
num_experiments = 3
batch_size = 25
num_acquisitions = 2000

In [0]:
results_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/UTKFace/AL_Results_Start_150/"
#Iterating across experiments, each of which has a different initial training set
for e in range(num_experiments):
  train_data_indices = list(np.load(trind_path+"trainindices"+str(e+1)+".npy"))
  pool_indices = [i for i in range(y_train_all.shape[0]) if i not in train_data_indices]
  mae_file = "RandMAEBS"+str(batch_size)+"_Ind"+str(e+1)+".npy"
  mse_file = "RandMSEBS"+str(batch_size)+"_Ind"+str(e+1)+".npy"
  ind_file = "RandIndBS"+str(batch_size)+"_Ind"+str(e+1)+".npy"
  exp_mse = []
  exp_mae = []
  #Commented code below used in case of resuming an active learning experiment rather than starting at the beginning
  #exp_mse = list(np.load(orig_results_path+mse_file))
  #exp_mae = list(np.load(orig_results_path+mae_file))
  #train_data_indices = list(np.load(orig_results_path+ind_file))
  #pool_indices = [i for i in range(y_train_all.shape[0]) if i not in train_data_indices]
  
  #Recording initial MSE and MAE
  [mae, mse] = run_model(X_train_all[train_data_indices,:,:,:], y_train_all[train_data_indices])
  exp_mse.append(mse)
  exp_mae.append(mae)
  
  #Looping over active learning iterations
  for acq in range(num_acquisitions//batch_size):
    #Random acquisition of new points
    new_train_ind = np.random.choice(pool_indices, batch_size, replace=False)
    for ind in new_train_ind:
      train_data_indices.append(ind)
      pool_indices.remove(ind)
    #Model training; evaluating model on test data and saving results
    [mae, mse] = run_model(X_train_all[train_data_indices,:,:,:], y_train_all[train_data_indices])
    exp_mae.append(mae)
    exp_mse.append(mse)
    print('Exp: ' + str(e+1) + ", # Training Pts: " + str(len(train_data_indices))+", Last 2 MSE: " + str(exp_mse[-2:]) + ", Len MSE: " + str(len(exp_mse)))

    np.save(results_path+mse_file,np.array(exp_mse))
    np.save(results_path+mae_file,np.array(exp_mae))
    np.save(results_path+ind_file,np.array(train_data_indices))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


Exp: 1, # Training Pts: 175, Last 2 MSE: [284.4525930087239, 301.133230838234], Len MSE: 2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


...

Exp: 2, # Training Pts: 2125, Last 2 MSE: [134.8700436137376, 153.21671697414467], Len MSE: 80


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


Exp: 2, # Training Pts: 2150, Last 2 MSE: [153.21671697414467, 133.78206729105932], Len MSE: 81


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


KeyboardInterrupt: ignored